# MySQL 8.0.40

## 0) 前提

* エンジン: **MySQL 8**
* 並び順: 任意（`ORDER BY` なし）
* `NOT IN` は NULL 罠のため回避
* 判定は **ID 基準**、表示は仕様どおりの列名と順序

## 1) 問題

* `RED という会社に関する注文を一度も担当していない営業担当者（SalesPerson）の name を求めよ。`
* 入力テーブル例: `SalesPerson(sales_id, name, salary, commission_rate, hire_date)`, `Company(com_id, name, city)`, `Orders(order_id, order_date, com_id, sales_id, amount)`
* 出力仕様: `name` のみ（任意順）

## 2) 最適解（単一クエリ）

> 本件はウィンドウ不要。**前処理 CTE で RED の注文に関与した sales_id をユニーク化**し、**左外部結合の反結合**で除外します。

```sql
WITH red_company AS (
  SELECT com_id
  FROM Company
  WHERE name = 'RED'
),
red_sales AS (
  -- RED へ紐づく注文を担当した営業の ID をユニーク化
  SELECT DISTINCT o.sales_id
  FROM Orders AS o
  JOIN red_company AS rc
    ON rc.com_id = o.com_id
)
SELECT s.name
FROM SalesPerson AS s
LEFT JOIN red_sales AS r
  ON r.sales_id = s.sales_id
WHERE r.sales_id IS NULL;

Runtime 1256 ms
Beats 72.00%

```

## 3) 代替解

> **`NOT EXISTS` による相関サブクエリ**。`NOT IN` を使わないため、NULL に安全。

```sql
SELECT s.name
FROM SalesPerson AS s
WHERE NOT EXISTS (
  SELECT 1
  FROM Orders AS o
  JOIN Company AS c
    ON c.com_id = o.com_id
  WHERE c.name = 'RED'
    AND o.sales_id = s.sales_id
);

Runtime 1189 ms
Beats 88.47%

```

## 4) 要点解説

* **反結合パターン**：`LEFT JOIN ... IS NULL` か `NOT EXISTS` が定石。NULL を含む可能性があるため `NOT IN` は避ける。
* **ユニーク化**：`DISTINCT` により同一営業が複数回 RED 注文を持つ重複を除去（結合行数を削減）。
* **インデックス**：`Company(name)`, `Orders(com_id, sales_id)`, `SalesPerson(sales_id)` があると高速。

## 5) 計算量（概算）

* `red_sales` 抽出：`Orders` に対して `com_id` で絞り込み → **O(N)**（索引あり想定）
* 反結合：`SalesPerson` と `red_sales` の結合 → **O(M)** 近似（`sales_id` インデックス前提）

## 6) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[Company から RED の com_id を取得] --> B[Orders を com_id で絞り RED に関与した sales_id をユニーク化]
  B --> C[SalesPerson と LEFT JOIN]
  C --> D[結合相手が NULL の行だけ抽出]
  D --> E[出力 name]
```
